# Web Scraping Assignment 3 

#### Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

In [2]:
# Activating the chrome browser
driver = webdriver.Edge(r"C:\Users\laxmi\FlipRobo\edgedriver_win64\msedgedriver.exe")

In [3]:
# Opening the homepage of Amazon.in
driver.get('https://www.amazon.in/')
# Asking the user to input the keywords he/she wants to search
user_inp = input('Enter the product you want to search : ')

Enter the product you want to search : guiter


In [4]:
search_bar = driver.find_element_by_id("twotabsearchtextbox")    # Locating searc_bar by id
search_bar.clear()                                               # clearing search_bar
search_bar.send_keys(user_inp)                                   # sending user input to search bar
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')       # Locating search_button by xpath
search_button.click()                                                                # Clicking the button to start search

#### Q2. 2.	In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
start_page = 0
end_page = 6
urls = []
for page in range(start_page,end_page):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.
Product urls of page 5 has been scraped.


In [6]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Name']=[]
prod_dict['Rating']=[]
prod_dict['No. of ratings']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

In [7]:
for url in urls:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver.find_element_by_xpath('//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver.find_element_by_xpath('//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver.find_element_by_xpath('//span[@id="acrPopover"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver.find_element_by_xpath('//a[@id="acrCustomerReviewLink"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    try:
        price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

Scraping URL =  https://www.amazon.in/Musical-Instruments/b/ref=sxts_spkl_4_0_460c4d37-38ea-441a-8493-97d687634812?ie=UTF8&node=3677697031&pd_rd_w=uf9FS&pf_rd_p=460c4d37-38ea-441a-8493-97d687634812&pf_rd_r=H4H3VKG14DC5FPSXFH17&pd_rd_r=a7a2836d-5264-4627-99fe-a52d762f23a8&pd_rd_wg=pxRoV&qid=1621960650
Scraping URL =  https://www.amazon.in/Juarez-Acoustic-Cutaway-038C-Strings/dp/B017NPCSLI/ref=sr_1_1?dchild=1&keywords=guiter&qid=1621960650&smid=A14CZOWI0VEHLG&sr=8-1
Scraping URL =  https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-Sunburst-Strings/dp/B01FYYTDS6/ref=sr_1_2?dchild=1&keywords=guiter&qid=1621960650&sr=8-2
Scraping URL =  https://www.amazon.in/Intern-INT-38C-Acoustic-Guitar-Black/dp/B073B6TQQY/ref=sr_1_3?dchild=1&keywords=guiter&qid=1621960650&sr=8-3
Scraping URL =  https://www.amazon.in/Juarez-Acoustic-Guitar-Cutaway-Strings/dp/B076QHZ4HZ/ref=sr_1_4?dchild=1&keywords=guiter&qid=1621960650&sr=8-4
Scraping URL =  https://www.amazon.in/Intern-38-inch-Cutaway-Design-Acoustic/dp/B07X4

Scraping URL =  https://www.amazon.in/Ketostics-Special-sunbrust-Acoustic-plectrums/dp/B07W3TB6T8/ref=sr_1_53?dchild=1&keywords=guiter&qid=1621960654&sr=8-53
Scraping URL =  https://www.amazon.in/Juarez-JRZ38C-ASH-Strings-Acoustic/dp/B072JTPNZC/ref=sr_1_54?dchild=1&keywords=guiter&qid=1621960654&sr=8-54
Scraping URL =  https://www.amazon.in/VD-ENTERPRISE-Adjustable-Classical-Instrument/dp/B08S323L6W/ref=sr_1_55?dchild=1&keywords=guiter&qid=1621960654&sr=8-55
Scraping URL =  https://www.amazon.in/SG-Musical-Acoustic-Guitar-Bag/dp/B083GYCV2C/ref=sr_1_56?dchild=1&keywords=guiter&qid=1621960654&sr=8-56
Scraping URL =  https://www.amazon.in/Juarez-Strings-Acoustic-Cutaway-Zebrawood/dp/B072Q9L8DG/ref=sr_1_57?dchild=1&keywords=guiter&qid=1621960654&sr=8-57
Scraping URL =  https://www.amazon.in/Fender-String-Cutaway-Acoustic-Polishing/dp/B07ZJNZ4Q6/ref=sr_1_58?dchild=1&keywords=guiter&qid=1621960654&sr=8-58
Scraping URL =  https://www.amazon.in/OOM-AKSHAR-Childrens-Instrument-Educational/dp/B0

Scraping URL =  https://www.amazon.in/Cavani-Acoustic-Guitar-rosewood-fretboard/dp/B07NVBJH1V/ref=sr_1_97?dchild=1&keywords=guiter&qid=1621960660&sr=8-97
Scraping URL =  https://www.amazon.in/FS100C-6-Strings-Acoustic-Plectrums-Natural/dp/B085MDF8D4/ref=sr_1_98?dchild=1&keywords=guiter&qid=1621960660&sr=8-98
Scraping URL =  https://www.amazon.in/Givson-G150SP-BKM-6-String-Cutaway-Acoustic/dp/B01D9OKF94/ref=sr_1_99?dchild=1&keywords=guiter&qid=1621960660&sr=8-99
Scraping URL =  https://www.amazon.in/Kadence-Frontier-Acoustic-Equalizer-Strings/dp/B01HRIHH9E/ref=sr_1_100?dchild=1&keywords=guiter&qid=1621960660&sr=8-100
Scraping URL =  https://www.amazon.in/SG-Musical-SGM-06-Silicon-Fingertip/dp/B083Q8ZFQ3/ref=sr_1_101?dchild=1&keywords=guiter&qid=1621960660&sr=8-101
Scraping URL =  https://www.amazon.in/Santkrupa-4-String-Acoustic-Learning-Adjustable/dp/B084MHQXGP/ref=sr_1_102?dchild=1&keywords=guiter&qid=1621960660&sr=8-102
Scraping URL =  https://www.amazon.in/RetroStyle-4-String-Acoust

KeyboardInterrupt: 

In [8]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df

,Brand,Name,Rating,No. of ratings,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,-,-,-,-,-,-,-,-,-,https://www.amazon.in/Musical-Instruments/b/re...
1,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5 stars,"9,351 ratings",-,7 Days Replacement,-,In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
2,Visit the JUAREZ Store,"Juarez JRZ38C Acoustic Guitar, 38 Inch Cutaway...",4.0 out of 5 stars,"3,202 ratings","₹ 2,499.00",7 Days Replacement,-,In stock.,"Black glossy finish, number of frets: 18, acou...",https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...
3,Visit the Intern Store,"Intern INT-38C Acoustic Guitar Kit, With Bag, ...",4.0 out of 5 stars,"5,308 ratings",-,7 Days Replacement,-,In stock.,Great looks with an innovative design to produ...,https://www.amazon.in/Intern-INT-38C-Acoustic-...
4,Visit the JUAREZ Store,"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...",3.9 out of 5 stars,788 ratings,"₹ 2,499.00",7 Days Replacement,-,In stock.,"Jumbo Design, 38 Inch Acoustic Steel String Gu...",https://www.amazon.in/Juarez-Acoustic-Guitar-C...
...,...,...,...,...,...,...,...,...,...,...
130,Brand: Generic,Signature 38 Inch Acoustic Guitar Kit Inbuilt ...,-,-,"₹ 5,000.00",7 Days Replacement,-,In stock.,"Material: Top: linden wood, Side & Back: - lin...",https://www.amazon.in/Signature-Acoustic-Guita...
131,Brand: verior,verior | worlds guitar toys for kids with micr...,-,-,-,7 Days Replacement,-,,3 aa batteries required not included || Incl...,https://www.amazon.in/verior-worlds-guitar-mic...
132,Brand: KETOSTICS,Ketostics Venus Special Acoustic Guitar with C...,3.9 out of 5 stars,27 ratings,-,7 Days Replacement,-,,Venus Special 6 Strings | Acoustic Guitar | Co...,https://www.amazon.in/Givson-Guitar-Special-St...
133,Visit the Fender Store,Fender SA105CE Electric Acoustic Guitar Dreadn...,5.0 out of 5 stars,4 ratings,-,7 Days Replacement,-,,Fender SA-105CE Dreadnought Acoustic Guitar Wi...,https://www.amazon.in/Fender-Electric-Acoustic...


In [ ]:
#saving data to csv
prod_df.to_csv('Amazon_{}.csv'.format(user_inp))

#### Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [31]:
# Opening images.google.com
driver.get('https://images.google.com/')

In [32]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")       # Inputing "banana" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [33]:
print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

start scrolling to generate more images on the page...


In [37]:
 images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

In [38]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

180

In [42]:
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("H:/Flip ROBO/banana/img"+str(i)+".jpg", "wb")
    file.write(response.content)

#### Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [67]:
# Asking for user input
prod = input(" Enter the name of the mobile phone you want to search : ")
driver.get('https://www.flipkart.com/')
time.sleep(3)
try:
    login_X_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')                      # Button to close login popup
    login_X_button.click()
except NoSuchElementException : 
    print("No Login page")
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')    # Finding the search bar using it's xpath
search_bar.clear()               # Clearing the search bar
search_bar.send_keys(prod)       # Inputing keyword to search
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

 Enter the name of the mobile phone you want to search : pixel


In [68]:
# Fetching urls of phones coming on 1st page
flip_urls = []
urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    flip_urls.append(url.get_attribute("href"))

In [69]:
len(flip_urls)

24

In [70]:
flip_dict = {}
flip_dict["Brand"] = []
flip_dict["Smartphone"] = []
flip_dict["Colour"] = []
flip_dict["RAM"] = []
flip_dict["Storage(ROM)"] = []
flip_dict["Primary Camera"] = []
flip_dict["Secondary Camera"] = []
flip_dict["Display Size"] = []
flip_dict["Display Resolution"] = []
flip_dict["Processor"] = []
flip_dict["Processor Cores"] = []
flip_dict["Battery Capacity"] = []
flip_dict["Battery Type"] = []
flip_dict["Price"] = []
flip_dict["URL"] = []

In [71]:
# Scraping data from each url
for url in flip_urls:
    driver.get(url)    # Saving url                                                     
    print("Scraping URL = ", url)
    flip_dict['URL'].append(url)                                                          # Loading the webpage by url
    time.sleep(2)
    
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
    
    try:
        brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      # Extracting Brand from xpath
        flip_dict["Brand"].append(brand.text.split()[0])
    except NoSuchElementException:
        flip_dict['Brand'].append('-')
        
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Brand from xpath
        flip_dict['Price'].append(price.text)
    except NoSuchElementException:
        flip_dict['Price'].append('-')
        
    try:
        name = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      # Extracting Brand from xpath
        flip_dict['Smartphone'].append(name.text)
    except NoSuchElementException:
        flip_dict['Smartphone'].append('-')
    
    try:
        color = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      # Extracting Name from xpath
        flip_dict['Colour'].append(color.text)
    except NoSuchElementException:
        flip_dict['Colour'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  # Extracting Ratings from xpath
        flip_dict['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        flip_dict['Display Size'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_res = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')     # Extracting no. of Ratings from xpath
        flip_dict['Display Resolution'].append(disp_res.text)
    except NoSuchElementException:
        flip_dict['Display Resolution'].append('-')
    
    try:
        pro_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_chk.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   # Extracting Price from xpath
        flip_dict['Processor'].append(processor.text)
    except NoSuchElementException:
        flip_dict['Processor'].append('-')
    
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_chk.text != "Processor Core" :
            core_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_chk.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        flip_dict['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        flip_dict['Processor Cores'].append('-')
    
    try:
        rom = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         # Extracting Expected Delivery from xpath
        flip_dict['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        flip_dict['Storage(ROM)'].append('-')
    
    try:
        ram = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['RAM'].append(ram.text)
    except NoSuchElementException:
        flip_dict['RAM'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        pri_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        flip_dict['Primary Camera'].append(pri_cam.text)
    except NoSuchElementException:
        flip_dict['Primary Camera'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        cam_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        flip_dict['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        flip_dict['Secondary Camera'].append('-')
        
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Capacity'].append(bat_cap.text)
    except NoSuchElementException:
        flip_dict['Battery Capacity'].append('-')
    
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[2]/ul/li')
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
            if bat_chk.text != "Battery Type" : raise NoSuchElementException
            bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Type'].append(bat_typ.text)
    except NoSuchElementException:
        flip_dict['Battery Type'].append('-')
    
                                                               

Scraping URL =  https://www.flipkart.com/google-pixel-3a-clearly-white-64-gb/p/itmfgk4jfgstaack?pid=MOBFFGFPJSCEXMSG&lid=LSTMOBFFGFPJSCEXMSGODGRZE&marketplace=FLIPKART&srno=s_1_1&otracker=search&otracker1=search&fm=SEARCH&iid=a5e3e22a-6fdf-4ba3-b771-a604ebad02a9.MOBFFGFPJSCEXMSG.SEARCH&ppt=sp&ppn=sp&ssid=6lybxosh2o0000001605705507073&qH=ab4086ecd47c568d
Scraping URL =  https://www.flipkart.com/google-pixel-3a-just-black-64-gb/p/itmfgk4jfgstaack?pid=MOBFFGFP7UHHJUZU&lid=LSTMOBFFGFP7UHHJUZUW7182D&marketplace=FLIPKART&srno=s_1_2&otracker=search&otracker1=search&fm=SEARCH&iid=a5e3e22a-6fdf-4ba3-b771-a604ebad02a9.MOBFFGFP7UHHJUZU.SEARCH&ppt=sp&ppn=sp&ssid=6lybxosh2o0000001605705507073&qH=ab4086ecd47c568d
Scraping URL =  https://www.flipkart.com/lg-g8x-black-128-gb/p/itme8a4f5f473aa4?pid=MOBFZKQWFRFMHKQK&lid=LSTMOBFZKQWFRFMHKQKVOP4L4&marketplace=FLIPKART&srno=s_1_3&otracker=search&otracker1=search&fm=SEARCH&iid=a5e3e22a-6fdf-4ba3-b771-a604ebad02a9.MOBFZKQWFRFMHKQK.SEARCH&ppt=sp&ppn=sp&ssid=6

In [73]:
print(len(flip_dict["Brand"]), len(flip_dict["Smartphone"]), len(flip_dict["Processor"]), len(flip_dict["Price"]), len(flip_dict['URL']))

24 24 24 24 24


In [74]:
flip_df = pd.DataFrame.from_dict(flip_dict)
flip_df

,Brand,Smartphone,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Battery Type,Price,URL
0,Google,Pixel 3a,Clearly White,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,14.22 cm (5.6 inch),2220 x 1080 pixels,Qualcomm Snapdragon 670,Octa Core,3000 mAh,-,"₹30,999",https://www.flipkart.com/google-pixel-3a-clear...
1,Google,Pixel 3a,Just Black,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,14.22 cm (5.6 inch),2220 x 1080 pixels,Qualcomm Snapdragon 670,Octa Core,3000 mAh,-,"₹30,999",https://www.flipkart.com/google-pixel-3a-just-...
2,LG,G8X,Black,6 GB,128 GB,12MP + 13MP,-,16.26 cm (6.4 inch),2340 x 1080 pixel,Qualcomm Snapdragon™ 855 Octa-core (up to 2.84...,Octa Core,4000 mAh,lithium-ion,"₹39,990",https://www.flipkart.com/lg-g8x-black-128-gb/p...
3,Samsung,Galaxy M21,Raven Black,4 GB,64 GB,48MP + 8MP + 5MP,-,16.26 cm (6.4 inch),2340 x 1080$$Pixel,Exynos 9611-Octa Core processor,Octa Core,6000 mAh,lithium-ion,"₹15,982",https://www.flipkart.com/samsung-galaxy-m21-ra...
4,Redmi,Redmi Note 9 Pro,Interstellar Black,4 GB,128 GB,48MP + 8MP,-,16.94 cm (6.67 inch),2400 x 1080$$pixel,Qualcomm Snapdragon 720G,Octa Core,5020 mAh,lithium-ion,"₹15,199",https://www.flipkart.com/redmi-note-9-pro-inte...
5,Redmi,Redmi 9 Prime,Space Blue,4 GB,128 GB,13MP Rear Camera,-,16.59 cm (6.53 inch),2340 x 1080$$pixel,-,Octa Core,5020 mAh,-,"₹11,335",https://www.flipkart.com/redmi-9-prime-space-b...
6,Redmi,Redmi 9,Sky Blue,4 GB,64 GB,13MP + 8MP,8MP Front Camera,16.59 cm (6.53 inch),720 x 1600$$pixel,MediaTek Helio G35,-,5000 mAh,Li-Po,"₹9,672",https://www.flipkart.com/redmi-9-sky-blue-64-g...
7,Redmi,Redmi 9,Carbon Black,4 GB,64 GB,13MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),720 x 1600$$pixel,MediaTek Helio G35,Octa Core,5000 mAh,Li-Po,"₹9,750",https://www.flipkart.com/redmi-9-carbon-black-...
8,Karbonn,KX3,"Black,Red",32 MB,32 MB,0.3MP Rear Camera,-,4.57 cm (1.8 inch),128 x 160$$pixel,-,-,800 mAh,Li-ion,₹799,https://www.flipkart.com/karbonn-kx3/p/itm60cf...
9,Itel,VISION1,GRADATION GREEN,2 GB,32 GB,8MP Rear Camera,-,15.46 cm (6.088 inch),1560 x 720$$pixel,UNISOC SC9863A OCTA CORE,Octa Core,4000 mAh,LI POLY,"₹6,499",https://www.flipkart.com/itel-vision1-gradatio...


#### Q5. 5.	Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [77]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element_by_id("searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element_by_id("searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City Name : Bangalore
URL Extracted:  https://www.google.co.in/maps/place/Bengaluru,+Karnataka/@28.6031121,77.3668853,15z/data=!4m5!3m4!1s0x3bae1670c9b44e6d:0xf8dfc3e8517e4fe0!8m2!3d12.9715987!4d77.5945627
Latitude = 28.6031121, Longitude = 77.3668853


#### Q6.	Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [10]:
driver.get('https://trak.in/')

In [14]:
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [42]:
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []

In [43]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [ ]:
fund_df.to_csv("Indian Startups_Q2_2020.csv")

#### Q7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [8]:
driver.get('https://www.digit.in/')

In [9]:
#clickng on top 10 option 
top_10=driver.find_element_by_xpath("//div[@class='menu']/ul/li[3]/a")
top_10.click()

#clicking on laptops option
laptops=driver.find_element_by_xpath("//ul[@id='top10list']/li[2]")
laptops.click()

#best gaming laptops link
best_gaming=driver.find_element_by_xpath("//div[@id='laptops']/a[3]")
driver.get(best_gaming.get_attribute('href'))

#intialising lists
name = []
price = []
OS = []
display = []
processor = []
HDD = []
RAM = []
weight = []
dimension = []
GPU = []

#names
names=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)
    
#os
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
#display
displays=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)
    
#processor
processors=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)
processor

#memory
memories=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")#list of specificaion name
memories_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")#values of specifiations 
for i in range(len(memories)):
        if memories[i].text=='Memory':
            HDD.append(memories_spec[i].text.split('/')[0])
            RAM.append(memories_spec[i].text.split('/')[1])
        else:
            HDD.append('No details available')#append no details as value for memory is missing in some of the laptops
            RAM.append('No details available')#append no details as value for memory is missing in some of the laptops

#weight
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(weights)):
        if weights[i].text=='Weight':
            weight.append(weight_spec[i].text)
        
#dimension
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

#graphical processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)
        
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")#getting the url of full specs links
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
    
for i in full_specs:#iterating throug every laptops full specs' page
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(prices.text)
    except NoSuchElementException:#exception handling for no price details
        price.append("No details available")
        
df=pd.DataFrame({"Name":name,
                "Price":price,
                "OS":OS,
                "Display":display,
                "HDD":HDD,
                 "RAM":RAM,
                "processor":processor,
                "weight":weight,
                "Dimension":dimension,
                "Graphical processor":GPU})
df

,Name,Price,OS,Display,HDD,RAM,processor,weight,Dimension,Graphical processor
0,MSI GT76 TITAN DT 9SG,"379,990",WINDOWS 10 PRO,"17.3"" (UHD 3840X2160)",1 TB 2x Samsung PM961 MZVLW256HEHP (RAID 0),64 GBGB DDR4,INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080
1,ALIENWARE 17 AREA-51M,No details available,WINDOWS 10 PRO,"17.3"" (FHD (1920 X 1080))",1 TB PCIe SSD,32GB DDR4,INTEL 9TH GEN CORE I9-9900K | 5000 MHZ,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080
2,HP OMEN 15 2020,"120,489",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB NA,16 GBGB DDR4,INTEL CORE I7-10750H | NA,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti
3,ASUS ROG ZEPHYRUS S GX701GXR REVIEW,"299,990",WINDOWS 10 HOME,"17.3"" (1920 X 1080)",No details available,No details available,INTEL CORE I7 | 2.6 GHZ,2.6,39.9 x 27.2 x 1.87,GeForce RTX 2080
4,LENOVO LEGION Y540,"79,990",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",Up to 512 GB PCIe SSD or up to 2 TB HDD SSD + HDD,8GB DDR4,9TH GENERATION INTEL® CORE™ I7-9750H | NA,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060
5,ASUS ROG ZEPHYRUS G GA502,"79,990",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,16GB DDR4,AMD RYZEN 7 | 2.3 GHZ WITH TURBO BOOST UPTO 4 GHZ,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti
6,ASUS ROG ZEPHYRUS S GX531,"146,000",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512GB SSD,16 GBGB DDR4,INTEL® CORE™ I7-8750H | 2.2 GHZ,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q)
7,MSI GT83VR 7RE TITAN SLI,"349,990",WINDOWS 10 HOME 64 BIT,"18.4"" (1920 X 1080)",1 TB with 512 GB SSD SATA,64GB DDR4,INTEL CORE I7 (7TH GENERATION) | 3.8 GHZ,5.5,458 x 339 x 69,8GB GDDR5 Nvidia GeForce GTX 1070
8,ASUS TUF GAMING FX505,"46,990",WINDOWS 10 PRO,"15.6"" (1920 X 1080)",1 TB SATA,16GB DDR4,INTEL CORE I7 - 87750H | 2.2 GHZ,2.2,360.4 x 262.0 x 25.8,NMvidia GTX 1060 (6GB)
9,DELL G3,"65,990",WINDOWS 10 HOME PLUS,15.6 MP | NA,256 SSD + 1 TB HDD SSD + SATA,16GB DDR4,8TH GENERATION INTEL® CORE™ I7-8750H PROCESSOR...,2.53,22.7 X 380 X 258,NVIDIA® GeForce® GTX


In [ ]:
df.to_csv("Gaming laptops_digit.csv")